# Apoio

In [28]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, RandomizedSearchCV
from scipy.stats import randint
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

## Leitura da base e manipulação de variáveis

In [2]:
# Leitura do dataset com 50 dimensões
df = pd.read_pickle('../data/final/enem_data_final.pkl')
df.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50,resposta_llama,acerto_llamma,resposta_deepseek,acerto_deepseek
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[0.09772969, 0.013166125, -0.032748703, 0.0580...","[-0.04434625, 0.2105245, -0.18144351, 0.030336...","[0.12940213, 0.22066694, 0.0586984, 0.03139533...",0.251504,0.736054,0.426387,C,True,C,True
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[-0.018823402, 0.07832528, -0.016192827, 0.027...","[0.21008351, 0.5248949, 0.03625112, 0.01772637...","[0.078055575, 0.19092263, -0.0076030395, -0.03...",0.634785,0.504648,0.587013,D,True,D,True
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[-0.013062097, 0.038470805, -0.020605344, 0.05...","[0.16413634, -0.035050992, 0.08454134, 0.23227...","[0.1313074, 0.058539692, 0.0961813, 0.17813754...",0.655402,0.634760,0.763059,D,True,D,True
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[-0.042833664, -0.01434522, -0.002431296, 0.01...","[0.102909006, 0.06116425, -0.13846825, 0.33034...","[0.11364358, 0.071867414, 0.046681672, 0.18046...",0.485125,0.628863,0.766145,E,True,E,True
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[0.010493249, 0.11095436, 0.017243173, 0.05319...","[0.267512, -0.081234336, 0.08019033, 0.1294613...","[0.13863416, 0.16260158, -0.0043905806, 0.2472...",0.559584,0.707497,0.692311,C,True,C,True


In [3]:
# Manipulando variáveis
df_new = df[["enunciado_embbedings_word2vec_50",
               "acerto_deepseek",
                "acerto_llamma",
                "similaridade_enunciado_gabarito_50",
                "similaridade_enunciado_distratores_50",
                "similaridade_gabarito_distratores_50",
               'nu_param_B']]
df_new.rename(columns={'nu_param_B': 'y', "enunciado_embbedings_word2vec_50":'X'}, inplace=True)

/tmp/ipykernel_2467/4207216969.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns={'nu_param_B': 'y', "enunciado_embbedings_word2vec_50":'X'}, inplace=True)


In [4]:
# Removendo observações Nan
df_new = df_new[df_new['y'].notna()].reset_index(drop=True)

In [13]:
# Explodindo a coluna X em várias colunas
df_features = pd.DataFrame(df_new['X'].tolist(), index=df_new.index)
df_features.columns = [f'X{i+1}' for i in range(df_features.shape[1])]

# Criando dataset final
df_final = pd.concat([df_features, df_new.drop(['X', 'y'], axis=1)], axis=1)
df_final['acerto_llamma'] = df_final['acerto_llamma'].astype(float)
df_final['acerto_deepseek'] = df_final['acerto_deepseek'].astype(float)
df_final.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X46,X47,X48,X49,X50,acerto_deepseek,acerto_llamma,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50
0,0.097730,0.013166,-0.032749,0.058064,0.066659,0.055518,0.005533,-0.010803,0.124288,-0.010112,...,0.018974,0.037986,0.010234,-0.070827,-0.104570,1.0,1.0,0.251504,0.736054,0.426387
1,-0.018823,0.078325,-0.016193,0.027615,0.080563,-0.001443,-0.011481,-0.011694,0.046921,-0.053186,...,-0.019229,0.167333,0.101951,-0.028866,-0.109328,1.0,1.0,0.634785,0.504648,0.587013
2,-0.013062,0.038471,-0.020605,0.051806,0.062729,-0.029429,0.076300,0.081863,0.074962,0.029681,...,-0.042368,0.108510,0.100451,-0.141360,-0.054252,1.0,1.0,0.655402,0.634760,0.763059
3,-0.042834,-0.014345,-0.002431,0.015723,0.100305,0.023464,0.047759,0.088246,0.083725,-0.059424,...,-0.014302,0.038790,0.018908,-0.013034,-0.059200,1.0,1.0,0.485125,0.628863,0.766145
4,0.010493,0.110954,0.017243,0.053193,0.005365,-0.082108,0.063029,-0.078666,-0.008422,0.028746,...,0.004114,0.075874,0.088452,-0.058793,-0.005872,1.0,1.0,0.559584,0.707497,0.692311


# Modelagem

## Separando dados

In [14]:
# Separando os dados
X = df_final
y = df_new['y']


# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)
y = pd.Series(y)

In [15]:
# Dividir em treino e teste (para avaliação externa)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Validação Cruzada

In [20]:
# Instancia o regressor
rf = RandomForestRegressor(random_state=42)

In [22]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# 'neg_mean_squared_error' retorna negativo do MSE
grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    refit=True
)

grid.fit(X_train, y_train)

best_rf = grid.best_estimator_
print("Melhores parâmetros:", grid.best_params_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Melhores parâmetros: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}


# Avaliação

## Conjunto de treino

In [24]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = -cross_val_score(
    best_rf, X_train, y_train,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
rmse_train_cv = np.sqrt(np.mean(mse_scores))
print(f"RMSE CV (treino): {rmse_train_cv:.4f}")

RMSE CV (treino): 0.4579


## Conjunto de testes

In [27]:
y_pred = best_rf.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
rmse_test = math.sqrt(mse_test)
print(f"RMSE (teste): {rmse_test:.4f}")

RMSE (teste): 0.4976
